# Lab 8: Controls Part 2

### EECS 16B: Designing Information Devices and Systems II, Summer 2022

Updated by Mingyang Wang (2022)

Updated by Yi-Hsuan Shih (2021, 2022)

yihsuan@berkeley.edu 

Hossein Najafi (2021)

Peter Schafhalter (2019)

Written by Vivek Athalye and Andrew Blatner (2017)

## Table of Contents

* [Introduction](#intro)
* [Part 1: Analyzing a Turn](#part1)
* [Part 2: Implementing Turns](#part2)


## <span style="color:#ba190f"> You need to run the Python scripts one by one, or errors will show up as they rely on the variables defined sequentially!!

## <span style="color:#ba190f"> DO NOT include units when submitting your answers on Gradescope! ONLY include the numerical value rounded to the number of decimal places specified in each question, in the units specified in the question. DO NOT include letters, words, etc. If a question involves entering an answer that is not a numerical value, the format of the answer will be clearly specified in that question.

<a id='part0'></a>
# <span style="color:navy">Part 0: Introduction</span>
This week, we will first revisit controls. Recall that our control scheme for SIXT33N modeled each wheel in the following way:

$$d_R[i+1] = d_R[i] + \theta_R u_R[i] - \beta_R$$

The open-loop model has an input $u[n]$, corresponding to the PWM level, and outputs the distance for that wheel. We implemented a closed-loop controller by using the desired velocity and the difference between the wheel distances, $$\delta[i] = d_L[i] - d_R[i]$$.

In this phase, we will implement turns by modifying the closed-loop control scheme we developed to make SIXT33N move straight. For a refresher on our closed-loop controller, please refer to the Lab 7: Controls Part 1 lab note [here](https://www.eecs16b.org/lab/notes/lab_note7.pdf). There is no lab note for Lab 8 and everything you need is in this notebook!


## (Optional) Pre-Lab

### Complete the (optional) pre-lab assignment before doing the lab. For all students, submit your answers to the Gradescope assignment "Pre-Lab 8: Controls Part 2". Please complete Pre-Lab 8 by Sunday, July 24 at 11:59pm.


<a id='part1'></a>
## <span style="color:navy">Part 1: Analyzing a Turn</span>

### Materials
- [SIXT33N Module in Tinkercad](https://www.tinkercad.com/things/9iqVbk7U76W?sharecode=cX6UeYwwIP4zjE7PQak41_qCXvd39t6f5YuXOhXHv84) 
- Closed-loop control scheme from Lab 7

### We want to turn with a specific radius.  What open-loop control input would lead to turning?  
During open-loop control, we sought to drive the car straight at our desired velocity $v^*$.  

We used our model of the car ($\theta$, $\beta$) to calculate the open-loop control input which drives the car straight in the absence of perturbations:  

$$u^{OL}_L = \frac{v^*+\beta_L}{\theta} $$
$$u^{OL}_R = \frac{v^*+\beta_R}{\theta} $$

Is it possible to calculate an open-loop control input which would result in turning with a desired radius?


### We have been turning during Closed-Loop Control
We can get a clue about turning by considering how we did closed-loop control. We perform feedback control of SIXT33N in order to minimize the difference between the two wheels ($\delta[i] = d_L[i] - d_R[i]$). When perturbations cause one wheel to get ahead of the other resulting in non-zero $\delta$, feedback control turns the car to correct the error.  

Let's walk through this.  

Our feedback control policy is: 
$$u_L[i] = u_L^{OL} - \frac{f_L}{\theta_L}\delta[i]$$

$$u_R[i] = u_R^{OL} + \frac{f_R}{\theta_R}\delta[i]$$


Let's say the right wheel has moved further than the left ($d_R[i] > d_L[i]$), resulting in a negative $\delta[i]$. A negative $\delta[i]$ results in a reduction of $u_R[i]$, and an increase of $u_L[i]$. But what does this mean physically?

If the right wheel is getting ahead of the left one, the left wheel will need to move a little faster and the right wheel a little slower for the wheels to even back out. Thus, feedback control corrects errors by turning to compensate for them.  

So $\delta \ne 0$ can be used to turn the car. Let's get more precise.  


### Turning via reference tracking

We would like the car to turn with a specified radius $r$ and linear velocity $v^*$. The controller's unit for distance is encoder ticks, but each tick is approximately 1 cm of wheel circumference.

To turn, we need to manually add some $\delta_{ref}$ to our $\delta$ value in every time step during the turn. By doing so, we are tricking our control scheme into thinking that our car has turned by some amount and generated that extra $\delta_{ref}$ of error, and cause it to turn in the other direction to compensate. Without loss of generality, we'll analyze a right turn, corresponding to adding a **negative** $\delta_{ref}$ value (we trick the car into thinking it has turned left, so it turns right as "compensation"). For a left turn, we simply add in a **positive** $\delta_{ref}$ instead. Our goal is to generate this $\delta_{ref}$ from the desired $r$ and $v^*$ for the controller to follow. It will be a function of the controller's time-step, as we want our car to turn gradually, rather than pivot turn.

Use the following variables:
- $i$ - time-step
- $r$ [cm] - turn radius of the center of the car; 1 cm ~ 1 encoder tick
- $d$ [ticks] - distance traveled by the center of the car
- $l$ [cm] - distance between the centers of the wheels; 1 cm ~ 1 encoder tick
- $\omega$ [rad/tick] - angular velocity
- $\theta$ [rad] - angle traveled

Inspect the following diagram:

<center>
<img width="400px" src="images/turning_figure.png">
</center>



From this geometry, can you write $\delta_{ref}[i]$ in the following form?
$$\delta_{ref}[i] = f(r,v^*,l,i)$$


<span style="color:#075a04"> **1. What is the relationship between $\delta_{ref}[i]$ and $v^*$ if $l$ and the turning radius $r$ are constant? <span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **2. How does $\delta_{ref}[i]$ change as a function of the number of time-steps ($i$) if the car is driving at a constant $v^*$ if $l$ and the turning radius $r$ are kept constant? <span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

 <span style="color:#075a04"> **3. If the car's turning radius doubles and its speed is also increased by 50%, how does the rate of $\delta_{ref}[i]$ compare to before? Find the ratio of $\LARGE \frac{\delta_{ref, new}[i]}{\delta_{ref, old}[i]}$.Enter a numerical value with two decimal places (e.g. 3.14).<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<a id='part2'></a>
## <span style="color:navy">Part 2: Implementing Turns</span>

### Materials
- Tinkercad Module
- Closed-loop control scheme from Lab 7
- Derivation from Part 1

### Sampling periods 
 
In our implementation, the control loop and the data collection have different periods, $T_c$ and $T_d$ respectively, and therefore different sampling frequencies, $F_c$ and $F_d$ respectively.  $F_c$ is an integer multiple of $F_d$: $m=\frac{F_c}{F_d} \in \mathbb{Z}^{+}$

To ensure the car is travelling at the same ground velocity in both cases, we want to have a constant distance, measured in ticks, travelled per second, or $\frac{ticks}{s}$.

$F_c$, with units [$\frac{sample_c}{s}$], is the number of times the closed-loop is sampled every second, and $F_d$, with units [$\frac{sample_d}{s}$], is the number of times data collection was sampled per second. Let $v^*_c$ be the velocity of closed-loop, and $v^*$ be the velocity of the data collection. $v^*_c$ has the units of [$\frac{ticks}{sample_c}$] and $v^*$ has units [$\frac{ticks}{sample_d}$].

To get a constant $\frac{ticks}{s}$, we need $v^*_c * F_c = v^* * F_d$.

Solving for $v^*_c$, we get $v^*_c = v^* * \frac{F_d}{F_c} = \frac{v^*}{m}$

We use $T_c=100$ms, $T_d=500ms$, so $F_c=10$, $F_d=2$, so $m=5$.

**<span style='color:red'> In your delta_reference function, use $\frac{v^*}{m}$ instead of $v^*$. Do not replace the original $v^*$ parameter defined in `CODE BLOCK CON1`!</span>**

### Implementation
- Open `turning.ino`.
- Specify number of commands according to `run_times` and `drive_modes` in `PREPROGRAMMED PATH` in line 37.
- We have already filled out **`CODE BLOCK CON1`** for you to make sure that there is no error propagation from the previous lab. Please fill out **`CODE BLOCK CON2`**.
- **Complete the function `delta_reference` in the `PREPROGRAMMED PATH`**.
    - Look at `CODE BLOCK CON0` to see how this is used!
    - Use the variables `TURN_RADIUS`, `v_star`, `CAR WIDTH`, and `i` (time step).

When you're done, try it out! SIXT33N should go straight, turn left, go straight, and turn right. To show how the car moves in TinkerCad, set **<span style='color:red'> mode = 0 </span>** in line 43 and observe the two waveforms being plotted in the serial plotter. The blue curve corresponds to the x-position of the car, and the orange line corresponds to the y-position of the car. The trajectory being represented is the shape below. The x and y positions of this trajectory are also shown over time. You should be able to observe a similar trajectory in the serial plotter in TinkerCad.

In the Serial plotter, the <span style="color:blue">BLUE</span> line corresponds to the <span style="color:blue">X</span> position of the car and the <span style="color:#EACE09">YELLOW</span> line to the <span style="color:#EACE09">Y</span> position of the car. (note: your graph will not look exactly the same)

<img width=700px src="images/car.png">
<img width=700px src="images/car2.png">

### Visualization of the Trajectory 
To visualize the trajectory, we are going to copy the simulation data from the serial monitor in TinkerCad, save it as txt file and then process it in the following code block. Please follow the steps as below.

- Set `mode=1` on line 43 in `turning.ino`. In this mode, the serial monitor would only print the value every `SAMPLE_CONST` of the sample. It is set to 10 to allow all the sampled data from 4 commands be printed in the serial monitor at the end of the simulation. A large number `10000, 10000` is inserted at the end of the commands so that we can recognize the end of the command in the python code. 
- After running the simulation, copy the data from the serial monitor and save to `trajactory.txt` file in the folder where you're running this Jupyter notebook. Then, run the following code block to observe the trajectory of the car.

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def plot_car_traj(file_name, command):
    color = {'DRIVE_STRAIGHT': 'blue', 'DRIVE_LEFT': 'orange', 'DRIVE_RIGHT': 'green'}
    data = np.genfromtxt(file_name, dtype=np.float, delimiter=",")
    
    fig = plt.figure(figsize=(10, 7))

    x = data[:, 0]
    y = data[:, 1]
    
    dx, _ = np.where(data==[10000, 10000])
    dx = [dx[2 * idx] for idx in range(dx.shape[0] // 2)]
    
    ax = fig.add_subplot(111)

    N = x.shape[0]
    ax.set_aspect('equal', adjustable='box')
    plt.xlabel("x")
    plt.ylabel("y")
    
    for idx in range(len(dx)):
        start_idx = 0 if idx == 0 else dx[idx - 1] + 1
        end_idx = -3 if idx == len(dx)-1 else dx[idx] - 1
        end_point = -2 if idx == len(dx)-1 else dx[idx] + 1
        ax.quiver(x[start_idx:end_idx], y[start_idx:end_idx],  np.append(x[start_idx + 1:end_idx], x[end_point])-x[start_idx:end_idx],  np.append(y[start_idx + 1:end_idx], y[end_point])-y[start_idx:end_idx], scale_units='xy', angles='xy', scale=1, color=color[command[idx]])
    plt.grid()
    plt.legend(command, loc="best")
    plt.title('Trajectory of the car')
    plt.show()
    print(f'final position (x, y): ({x[-2]}, {y[-2]})')
    return 



In [ ]:
plot_car_traj('trajectory.txt', ('DRIVE_STRAIGHT', 'DRIVE_LEFT', 'DRIVE_STRAIGHT', 'DRIVE_RIGHT'))

<span style="color:#075a04"> **4. If the car's speed $v^*$ is set to be $185.326$, and both $f_{left}$ and $f_{right}$ are set to 0.15, what is the final value of X position of the car? The coordinates being printed in the serial plotter are formatted as (X position, Y position). Enter a numerical value with two decimal places (e.g. 3.14).<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **5. In the same case for Q4, what is the final value of Y position of the car? Enter a numerical value with two decimal places (e.g. 3.14).<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **6. Change the length of the 1st command from 8000 to 16000. You can do so by changing the line `int run_times[NUM_COMMANDS] = {8000, 25200, 8000, 25200};` to `int run_times[NUM_COMMANDS] = {16000, 25200, 8000, 25200};`. If the car's speed $v^*$ is set to be $185.326$, and both $f_{left}$ and $f_{right}$ are set to 0.15, what is the final value of the X position of the car?<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **7. In the same case as Q6, what is the final value of the Y position of the car? Enter a numerical value with two decimal places (e.g. 3.14).<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

**8. Revert the run_times back to {8000, 25200, 8000, 25200}. Switch the 2nd and 4th command. You can do so by changing the line `int drive_modes[NUM_COMMANDS] = {DRIVE_STRAIGHT, DRIVE_LEFT, DRIVE_STRAIGHT, DRIVE_RIGHT
};` to `int drive_modes[NUM_COMMANDS] = {DRIVE_STRAIGHT, DRIVE_RIGHT, DRIVE_STRAIGHT, DRIVE_LEFT};`. If the car's speed $v^*$ is set to be $185.326$, and both $f_{left}$ and $f_{right}$ are set to 0.15?**
    
<span style="color:#075a04"> **8a. What do you observe in the trajectory of the car? Which of the following applies to the trajectory the car is taking? *Hint: Run simulation at `mode = 1`, copy the data and paste it to `
trajectory_q8.txt` in this folder. You should be able to see the trajectory in the following code block.*<span style="color:#ba190f">** 

  < YOUR ANSWER ON GRADESCOPE >    

In [ ]:
plot_car_traj('trajectory_q8.txt', ('DRIVE_STRAIGHT', 'DRIVE_RIGHT', 'DRIVE_STRAIGHT', 'DRIVE_LEFT'))

<span style="color:#075a04"> **8b. What is the final value of the X position of the car? Enter a numerical value with two decimal places (e.g. 3.14).<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **8c. What is the final value of the Y position of the car? Enter a numerical value with two decimal places (e.g. 3.14).<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<a id='part3'></a>
# <span style="color:#ba190f">CHECKOFF</span>
-----

### Submit your answers to Questions 1-8 to the Gradescope assignment "Lab 8: Controls Part 2." Lab 8 is due on Sunday, July 31 at 11:59pm. No late submissions will be accepted.

### Make sure your circuits are saved properly in Tinkercad. You will need them in the next project phase!

### Remember what each part of your circuit is for by recording this information in a Google doc or somewhere else safe. You will need to write a summary for your final lab report.

### Save this notebook somewhere you can access!

We recommend making a shared Google Drive with your lab partner, since GitHub can be annoying with Jupyter notebooks. It's good to familiarize yourself with the user interface of Tinkercad and understand the basic functions of these circuits that you have built and analyzed. You will also need to report your design processes in the final lab report.


### Great job finishing 16B Lab 8!
